In [18]:
import numpy as np 
import pandas as pd
import operator
import os
import chart_studio.tools as tls

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile

import os
import numpy as np

from PIL import Image

from scipy.fftpack import fft
import librosa
import librosa.display

import plotly as py
tls.set_credentials_file(username='iam4tart',api_key='Docq9bPOpIqBxnDmPIxJ')

import chart_studio.plotly as py
import plotly.graph_objs as go

import cufflinks as cf
import IPython.display as ipd

%matplotlib inline

In [19]:
def get_files(path):
    data_ = []
    count = {}
    files = os.listdir(path)
    for file in files:
        labels = file.split("-")[:3]
        data = {
            "keyword": labels[0],
            "stress": labels[2],
            "environment": labels[1],
            "path": os.path.join(path, file)
        }
        for label in labels:
            if label not in count.keys():
                count[label] = 1
            else:
                count[label] += 1
        data_.append(data)
    if data_:
        df = pd.DataFrame(data_)
    else:
        df = pd.DataFrame(columns=['keyword', 'stress', 'environment', 'path'])
    return data_, count, df

path = 'EmoSpeech/'
dataset, count, df = get_files(path)
df_nopath = df.iloc[:, :-1]

print(count)
print()
print(df.head())
print("Total Number of samples: ", len(df))

{'go': 483, 'safe': 5098, 'angry': 623, 'background': 1107, 'calm': 3020, 'unknown': 1169, 'happy': 543, 'no': 539, 'fearful': 1162, 'help': 588, 'stop': 535, 'yes': 501, 'None': 157, 'bacho': 583, 'scream': 352, 'gunshots': 10, 'glass break': 45}

      keyword   stress environment  \
0          go    angry        safe   
1  background     calm        safe   
2     unknown    happy        safe   
3          no  fearful        safe   
4        help  fearful        safe   

                                                path  
0  EmoSpeech/go-safe-angry-20190731001623-d14a028...  
1  EmoSpeech/background-safe-calm-20190709093534-...  
2  EmoSpeech/unknown-safe-happy-1561964984.688572...  
3  EmoSpeech/no-safe-fearful-20190712140008-d14a0...  
4  EmoSpeech/help-safe-fearful-20190623190210-d14...  
Total Number of samples:  5505


In [20]:
def extract_features(file_path, duration=3, sr=16000, n_mfcc=13, max_frames=150):
    y, sr = librosa.load(file_path, sr=sr, duration=duration)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Pad or truncate to a fixed number of frames
    if mfccs.shape[1] < max_frames:
        mfccs = np.pad(mfccs, ((0, 0), (0, max_frames - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :max_frames]
    return mfccs.T  # Transpose to have time along the columns



In [21]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers

In [22]:


# # Encode labels
label_encoder = LabelEncoder()
df['stress_encoded'] = label_encoder.fit_transform(df['stress'])
df['environment_encoded'] = label_encoder.fit_transform(df['environment'])

# Split the dataset into training and test sets
X = np.array([extract_features(file) for file in df['path']])
y_stress = to_categorical(df['stress_encoded'])
y_env = to_categorical(df['environment_encoded'])
X_train, X_test, y_stress_train, y_stress_test, y_env_train, y_env_test = train_test_split(X, y_stress, y_env, test_size=0.2, random_state=42)

# # Define the neural network model
# input_shape = X_train.shape[1:]
# stress_output_classes = len(df['stress'].unique())
# env_output_classes = len(df['environment'].unique())

# model = models.Sequential([
#     layers.Input(shape=input_shape),
#     layers.LSTM(128, return_sequences=True),
#     layers.LSTM(128),
#     layers.Dense(stress_output_classes, activation='softmax', name='stress_output'),
#     layers.Dense(env_output_classes, activation='softmax', name='env_output')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, {'stress_output': y_stress_train, 'env_output': y_env_train}, validation_data=(X_test, {'stress_output': y_stress_test, 'env_output': y_env_test}), epochs=10)

# # Evaluate the model
# # Evaluate the model

# print(model.evaluate(X_test, {'stress_output': y_stress_test, 'env_output': y_env_test}))
# # loss, stress_loss, env_loss, stress_accuracy, env_accuracy = model.evaluate(X_test, {'stress_output': y_stress_test, 'env_output': y_env_test})



In [23]:
print(y_env)
print(y_stress)

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [24]:
print(df['stress'].unique(),"\n",df['environment'].unique())

['angry' 'calm' 'happy' 'fearful' 'None'] 
 ['safe' 'scream' 'gunshots' 'glass break']


In [25]:
# sample_path = 'House/go-scream-calm-20190713144915.wav'


# # Extract features from the sample
# sample_features = extract_features(sample_path)
# sample_features = np.expand_dims(sample_features, axis=0)

# # Predict stress and environment labels
# stress_pred, env_pred = model.predict(sample_features)[0]

# # Get the predicted labels
# predicted_stress_index = np.argmax(stress_pred)
# predicted_env_index = np.argmax(env_pred)
# predicted_stress = label_encoder.inverse_transform([predicted_stress_index])[0]
# predicted_env = label_encoder.inverse_transform([predicted_env_index])[0]

# print(f'Predicted Stress: {predicted_stress}, Predicted Environment: {predicted_env}')

In [40]:
# Define the neural network model
input_shape = X_train.shape[1:]
stress_output_classes = len(df['stress'].unique())
env_output_classes = len(df['environment'].unique())

model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(stress_output_classes, activation='softmax', name='stress_output'),
    layers.Dense(env_output_classes, activation='softmax', name='env_output')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
# Train the model
model.fit(X_train, {'stress_output': y_stress_train, 'env_output': y_env_train}, validation_data=(X_test, {'stress_output': y_stress_test, 'env_output': y_env_test}), epochs=10)

# Evaluate the model
print(model.evaluate(X_test, {'stress_output': y_stress_test, 'env_output': y_env_test}))

# Prediction example
sample_path = 'EmoSpeech/yes-scream-fearful-20190729211522-d14a028c2a3a2bc9476102bb288234c415a2b01f828ea62ac5b3e42f.wav'

# Extract features from the sample
sample_features = extract_features(sample_path)
sample_features = np.expand_dims(sample_features, axis=0)

# Predict stress and environment labels
# stress_pred, env_pred = model.predict(sample_features)
print(model.predict(sample_features)[0])

# # Get the predicted labels
# predicted_stress_index = np.argmax(stress_pred)
# predicted_env_index = np.argmax(env_pred)
# predicted_stress = label_encoder.inverse_transform([predicted_stress_index])[0]
# predicted_env = label_encoder.inverse_transform([predicted_env_index])[0]

# # Print predictions and class labels
# print(f'Stress Prediction: {stress_pred}')
# print(f'Environment Prediction: {env_pred}')
# print(f'Predicted Stress: {predicted_stress}, Predicted Environment: {predicted_env}')

# print(f'Predicted Stress: {predicted_stress}, Predicted Environment: {predicted_env}')


Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0209 - loss: 1.8314 - val_accuracy: 0.0572 - val_loss: 1.2563
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6161 - loss: 1.1733 - val_accuracy: 0.9310 - val_loss: 0.9511
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9308 - loss: 0.8919 - val_accuracy: 0.9310 - val_loss: 0.7396
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9255 - loss: 0.7036 - val_accuracy: 0.9310 - val_loss: 0.5969
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9267 - loss: 0.5755 - val_accuracy: 0.9310 - val_loss: 0.5030
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9277 - loss: 0.4897 - val_accuracy: 0.9310 - val_loss: 0.4411
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9260 - loss: 0.4369 - val_accuracy: 0.9310 - val_loss: 0.3996
Epoch 8/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9282 - loss: 0.3949 - val_accuracy: 0.

In [41]:
print(sample_features)

[[[-331.80505     83.01903     -0.8065515 ...   -5.7957993   -6.5359173
     -6.778347 ]
  [-286.3805      79.819916    -3.7666516 ...   -3.281063    -4.330223
     -9.153894 ]
  [-292.41898     83.718216    -6.4664483 ...   -4.276279    -2.1556718
     -9.537165 ]
  ...
  [   0.           0.           0.        ...    0.           0.
      0.       ]
  [   0.           0.           0.        ...    0.           0.
      0.       ]
  [   0.           0.           0.        ...    0.           0.
      0.       ]]]


In [39]:
# # # Get the predicted labels
# stress_pred, env_pred, _, _x = model.predict(sample_features)[0]
# predicted_stress_index = np.argmax(stress_pred)
# predicted_env_index = np.argmax(env_pred)
# predicted_stress = label_encoder.inverse_transform([predicted_stress_index])[0]
# predicted_env = label_encoder.inverse_transform([predicted_env_index])[0]

# # # Print predictions and class labels
# print(f'Stress Prediction: {stress_pred}')
# print(f'Environment Prediction: {env_pred}')
# print(f'Predicted Stress: {predicted_stress}, Predicted Environment: {predicted_env}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Stress Prediction: 0.8799220323562622
Environment Prediction: 0.05698821321129799
Predicted Stress: glass break, Predicted Environment: glass break


In [42]:
prob = model.predict(sample_features)
classes = np.argmax(prob, axis = 1)
print(classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[2]
